##### img_dataset_0818_add5x, txt_dataset_0818_add5x: add 5x answers, 
delete 320 samples which were flagged problematic by annotators (60 from img data, 260 from txt data)
##### img_dataset_0819_16neg, txt_dataset_0819_16neg: only keep 16 neg samples

In [1]:
import json, pickle, copy, os, time, random
from collections import Counter, defaultdict
import numpy as np
from pprint import pprint

In [2]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0818.json", "r"))
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0817_neg_ranked.json", "r"))

print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

print(Counter([img_dataset[k]['split'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset]))
print(len(set([img_dataset[k]['Guid'] for k in img_dataset])))

Counter({'train': 17812, 'test': 5084, 'val': 2455})
25351
Counter({'train': 16447, 'ood_test': 3987, 'val': 2511, 'ind_test': 2507})
Counter({'YesNo': 8429, 'Others': 6713, 'choose': 5240, 'number': 2339, 'color': 2069, 'shape': 662})
25452


In [3]:
print("img data, txt_negFacts: ", np.mean([len(img_dataset[k]['txt_negFacts']) for k in img_dataset]), np.max([len(img_dataset[k]['txt_negFacts']) for k in img_dataset]), np.min([len(img_dataset[k]['txt_negFacts']) for k in img_dataset]))
print("img data, img_negFacts: ", np.mean([len(img_dataset[k]['img_negFacts']) for k in img_dataset]), np.max([len(img_dataset[k]['img_negFacts']) for k in img_dataset]), np.min([len(img_dataset[k]['img_negFacts']) for k in img_dataset]))
print("img data, img_posFacts: ", np.mean([len(img_dataset[k]['img_posFacts']) for k in img_dataset]), np.max([len(img_dataset[k]['img_posFacts']) for k in img_dataset]), np.min([len(img_dataset[k]['img_posFacts']) for k in img_dataset]))

print("txt data, txt_negFacts: ", np.mean([len(txt_dataset[k]['txt_negFacts']) for k in txt_dataset]), np.max([len(txt_dataset[k]['txt_negFacts']) for k in txt_dataset]), np.min([len(txt_dataset[k]['txt_negFacts']) for k in txt_dataset]))
print("txt data, img_negFacts: ", np.mean([len(txt_dataset[k]['img_negFacts']) for k in txt_dataset]), np.max([len(txt_dataset[k]['img_negFacts']) for k in txt_dataset]), np.min([len(txt_dataset[k]['img_negFacts']) for k in txt_dataset]))
print("txt data, txt_posFacts: ", np.mean([len(txt_dataset[k]['txt_posFacts']) for k in txt_dataset]), np.max([len(txt_dataset[k]['txt_posFacts']) for k in txt_dataset]), np.min([len(txt_dataset[k]['txt_posFacts']) for k in txt_dataset]))

img data, txt_negFacts:  35.775656137042276 41 0
img data, img_negFacts:  29.54086122898004 44 7
img data, img_posFacts:  1.4371758604431872 2 1
txt data, txt_negFacts:  28.91597964577334 72 2
txt data, img_negFacts:  21.55271981381405 40 0
txt data, txt_posFacts:  2.0391306062877206 5 1


In [4]:
TestSet5Answers_0819 = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/TestSet5Answers_0819.json", "r"))
print(len(TestSet5Answers_0819))

11579


In [22]:
'''
tmp = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/TestSet5Answers_0818_from_Mridu.json", "r"))
print(len(tmp))
print(type(tmp))
TestSet5Answers_0819 = {}
for i in tmp:
    guid = i['Guid']
    TestSet5Answers_0819[guid] = copy.deepcopy(i)
print(len(TestSet5Answers_0819))
json.dump(TestSet5Answers_0819, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/TestSet5Answers_0819.json", "w"), indent=4)
'''

11579
<class 'list'>
11579


In [5]:
a = set([img_dataset[k]['Guid'] for k in img_dataset if img_dataset[k]['split'] in ['ind_test', 'ood_test']])
a = a.union(set([txt_dataset[k]['Guid'] for k in txt_dataset if txt_dataset[k]['split'] in ['test']]))
print(set(list(TestSet5Answers_0819.keys())) - a)
print(a - set(list(TestSet5Answers_0819.keys())))

{'dc4f2460d46f11ebba07a504a20e7724', 'dc4dc3b8d46f11ebba07a504a20e7724'}
{'dc4c05a0d46f11ebba07a504a20e7724'}


In [6]:
print(len([k for k in TestSet5Answers_0819 if TestSet5Answers_0819[k]['HasIssue']]))

320


In [8]:
### Need to modify the split a little bit. Some mismatch
move_to_test = ['dc4dc3b8d46f11ebba07a504a20e7724', 'dc4f2460d46f11ebba07a504a20e7724']
move_to_train = ['dc4c05a0d46f11ebba07a504a20e7724']
img_dataset_0818_add5x = copy.deepcopy(img_dataset)
txt_dataset_0818_add5x = copy.deepcopy(txt_dataset)

delete_img_count = 0
delete_txt_count = 0
total_count = 0
for k in list(img_dataset_0818_add5x.keys()):
    guid = img_dataset_0818_add5x[k]['Guid']
    if guid in move_to_test:
        print("img data k = {} move from {} to test".format(k, img_dataset_0818_add5x[k]['split']))
        img_dataset_0818_add5x[k]['split'] = 'ind_test'
    if guid in move_to_train:
        print("img data k = {} move from {} to train".format(k, img_dataset_0818_add5x[k]['split']))
        img_dataset_0818_add5x[k]['split'] = 'train'
    
    ori_A = img_dataset_0818_add5x[k]['A']
    img_dataset_0818_add5x[k]['A'] = [ori_A]
    
    if not guid in TestSet5Answers_0819: continue
    total_count += 1
    
    annotation_5x = TestSet5Answers_0819[guid]
    assert annotation_5x['Question'] in img_dataset_0818_add5x[k]['Q']
    if annotation_5x['HasIssue']: 
        print("delete img data k = {}".format(k))
        del img_dataset_0818_add5x[k]
        delete_img_count += 1
        continue
    
    for a in annotation_5x['Answers']:
        img_dataset_0818_add5x[k]['A'].append('"' + a + '"')

for k in list(txt_dataset_0818_add5x.keys()):
    guid = txt_dataset_0818_add5x[k]['Guid']
    if guid in move_to_test:
        print("txt data k = {} move from {} to test".format(k, txt_dataset_0818_add5x[k]['split']))
        txt_dataset_0818_add5x[k]['split'] = 'test'
    if guid in move_to_train:
        print("txt data k = {} move from {} to train".format(k, txt_dataset_0818_add5x[k]['split']))
        txt_dataset_0818_add5x[k]['split'] = 'train'
    
    ori_A = txt_dataset_0818_add5x[k]['A']
    txt_dataset_0818_add5x[k]['A'] = [ori_A]
    
    if not guid in TestSet5Answers_0819: continue
    total_count += 1
    
    annotation_5x = TestSet5Answers_0819[guid]
    assert annotation_5x['Question'] in txt_dataset_0818_add5x[k]['Q']
    if annotation_5x['HasIssue']: 
        print("delete txt data k = {}".format(k))
        del txt_dataset_0818_add5x[k]
        delete_txt_count += 1
        continue
    
    for a in annotation_5x['Answers']:
        txt_dataset_0818_add5x[k]['A'].append('"' + a + '"')

print("total_count = ", total_count)
print("delete_img_count = ", delete_img_count)
print("delete_txt_count = ", delete_txt_count)
print(len(img_dataset_0818_add5x))
print(len(txt_dataset_0818_add5x))

delete img data k = 6042
delete img data k = 6052
delete img data k = 6471
delete img data k = 6720
delete img data k = 7637
delete img data k = 7758
delete img data k = 8047
delete img data k = 8214
delete img data k = 3560
img data k = 4087 move from ind_test to train
delete img data k = 4511
delete img data k = 4629
delete img data k = 5415
delete img data k = 5462
delete img data k = 5572
delete img data k = 5995
delete img data k = 127
delete img data k = 191
delete img data k = 215
delete img data k = 256
delete img data k = 331
delete img data k = 399
delete img data k = 560
delete img data k = 561
delete img data k = 1328
delete img data k = 1611
delete img data k = 1721
delete img data k = 1927
delete img data k = 25308
delete img data k = 2038
delete img data k = 2154
delete img data k = 2179
delete img data k = 2241
delete img data k = 2244
delete img data k = 21640
delete img data k = 22121
delete img data k = 22371
delete img data k = 22411
delete img data k = 23090
delete

In [9]:
json.dump(img_dataset_0818_add5x, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0818_add5x.json", "w"), indent=4)
json.dump(txt_dataset_0818_add5x, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0818_add5x.json", "w"), indent=4)

In [11]:
img_dataset_0818_add5x = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0818_add5x.json", "r"))
txt_dataset_0818_add5x = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0818_add5x.json", "r"))

print(Counter([txt_dataset_0818_add5x[k]['split'] for k in txt_dataset_0818_add5x]))
print(len(set([txt_dataset_0818_add5x[k]['Guid'] for k in txt_dataset_0818_add5x])))

print(Counter([img_dataset_0818_add5x[k]['split'] for k in img_dataset_0818_add5x]))
print(Counter([img_dataset_0818_add5x[k]['Qcate'] for k in img_dataset_0818_add5x]))
print(len(set([img_dataset_0818_add5x[k]['Guid'] for k in img_dataset_0818_add5x])))

Counter({'train': 17812, 'test': 4824, 'val': 2455})
25091
Counter({'train': 16448, 'ood_test': 3948, 'val': 2511, 'ind_test': 2485})
Counter({'YesNo': 8410, 'Others': 6689, 'choose': 5226, 'number': 2337, 'color': 2068, 'shape': 662})
25392


In [15]:
img_dataset_0819_16neg = copy.deepcopy(img_dataset_0818_add5x)
for k in img_dataset_0819_16neg:
    img_dataset_0819_16neg[k]['img_negFacts'] = img_dataset_0819_16neg[k]['img_negFacts'][:16]
    img_dataset_0819_16neg[k]['txt_negFacts'] = img_dataset_0819_16neg[k]['txt_negFacts'][:16]
txt_dataset_0819_16neg = copy.deepcopy(txt_dataset_0818_add5x)
for k in txt_dataset_0819_16neg:
    txt_dataset_0819_16neg[k]['img_negFacts'] = txt_dataset_0819_16neg[k]['img_negFacts'][:16]
    txt_dataset_0819_16neg[k]['txt_negFacts'] = txt_dataset_0819_16neg[k]['txt_negFacts'][:16]
print(len(img_dataset_0819_16neg), len(txt_dataset_0819_16neg))
json.dump(img_dataset_0819_16neg, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0819_16neg.json", "w"), indent=4)
json.dump(txt_dataset_0819_16neg, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0819_16neg.json", "w"), indent=4)

25392 25091


In [14]:
img_dataset_0818_add5x['2']

{'A': ['"There are trees near the Bethesda Fountain which are taller than the fountain."'],
 'Guid': 'dc43c926d46f11ebba07a504a20e7724',
 'Keywords_A': '"Yes"',
 'Q': '"Are there any trees near the Bethesda Fountain which are taller than the fountain?"',
 'Qcate': 'YesNo',
 'Qtype': 'img-Singlehop',
 'img_negFacts': [{'caption': 'Angel of the Waters Fountain and Bethesda Terrace, Central Park, NYC',
   'image_id': '00003295',
   'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Angel_of_the_Waters_Fountain_and_Bethesda_Terrace%2C_Central_Park%2C_NYC.jpg/800px-Angel_of_the_Waters_Fountain_and_Bethesda_Terrace%2C_Central_Park%2C_NYC.jpg',
   'overlap_scores': '(0.3333, 0.3333, 0.3333, 0.119, 0.1471, 0.3133)',
   'title': 'Angel of the Waters Fountain and Bethesda Terrace, Central Park, NYC',
   'url': 'https://commons.wikimedia.org/wiki/File:Angel_of_the_Waters_Fountain_and_Bethesda_Terrace%2C_Central_Park%2C_NYC.jpg'},
  {'caption': 'Bethesda Fountain from the Bethesd